***Deriving Business Insights from Relational Database Management Systems with Natural Language***


In [ ]:
#Install required libraries
!pip install langchain openai mysql-connector-python pymysql plotly cohere tiktoken fastapi kaleido python-multipart uvicorn ydata-profiling -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Import required libraries
import os
import numpy as np
import pandas as pd
import mysql.connector
import plotly.express as px
import re

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_toolkits.sql.base import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from getpass import getpass
from ydata_profiling import ProfileReport

####**Setup Database, LLM & Agent Executor**

In [ ]:
# Function to Create a connection to the MySQL sample database in google cloud server
def get_db_connection(host, user, password, database):

    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

In [ ]:
# Function to  print table names from database in database connection
def show_tables(cursor):
    table_list = []
    try:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        if tables:
            for table in tables:
                table_list.append(table[0])
        else:
            print("No tables found in the database.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    return table_list


In [ ]:
# Create a connection to the MySQL sample database in google cloud and print table names
conn = get_db_connection(
    host="35.197.251.91",
    user="root",
    password="P@ssw0rd",
    database="classicmodels"
)

if conn:
        with conn.cursor() as cursor:
            table_names = show_tables(cursor)
            print("Tables in database:", table_names)
        conn.close()

Tables in database: ['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']


In [ ]:
# Function to Setup SQLAlchemy engine
def get_sqlalchemy_engine(DB_USER, DB_PASS, DB_NAME, DB_HOST, DB_PORT):

    connection_string = f'mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(connection_string)

    return engine


In [ ]:
# Setup SQLAlchemy engine
engine = get_sqlalchemy_engine(
    # database parameters
    DB_USER = "root",
    DB_PASS = quote_plus("P@ssw0rd"),
    DB_NAME = "classicmodels",
    DB_HOST = "35.197.251.91",
    DB_PORT = 3306
)

In [ ]:
# Function to Setup Langchain Agent_Executor
def setup_agent_executor(engine):

    # Create an SQLDatabase instance with your SQLAlchemy engine
    db = SQLDatabase(engine=engine)

    # Define llm
    #llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)

    # Define toolkit for langchain agent
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)

    #Create agent executor
    agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    handle_parsing_errors=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    output_parser=CommaSeparatedListOutputParser(),
    verbose=True
    )

    return agent_executor

In [ ]:
 #Set OpenAI Key
OPENAI_API_KEY = getpass("Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter OpenAI API Key: ··········


In [ ]:
# Setup Langchain Agent_Executor
agent_executor = setup_agent_executor(engine)

####**Natural Language Queries and Exploratory Data Analysis**

**Identify customers bringing in the most revenue**

In [ ]:
# List the top 10 highest paying customers and the total payments for each customer?
query = input ("Enter question :")

Enter question :List the top 10 highest paying customers and the total payments for each customer?


In [ ]:
response = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `customers, payments`
responded: The relevant tables for this query seem to be 'customers' and 'payments'. Let's check the schema of these tables.


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows

In [ ]:
print(response)

Here are the top 10 highest paying customers and their total payments:

1. Euro+ Shopping Channel: $715,738.98
2. Mini Gifts Distributors Ltd.: $584,188.24
3. Australian Collectors, Co.: $180,585.07
4. Muscle Machine Inc: $177,913.95
5. Dragon Souveniers, Ltd.: $156,251.03
6. Down Under Souveniers, Inc: $154,622.08
7. AV Stores, Co.: $148,410.09
8. Anna's Decorations, Ltd: $137,034.22
9. Corporate Gift Ideas Co.: $132,340.78
10. Saveley & Henriot, Co.: $130,305.35


In [ ]:
def parse_customer_payment_response(data_str):
    # Step 1: Data Preprocessing
    # Extract lines containing customer and payment information
    lines = [line.strip() for line in data_str.strip().split('\n') if '$' in line]

    # Step 2: Data Extraction
    # Extract customer names and their corresponding payments
    customer_names = []
    total_payments = []

    for line in lines:
        # Extract using regular expressions
        match = re.search(r'^\d+\.\s(.*?): \$(.*)$', line)
        if match:
            customer_name, total_payment = match.groups()
            # Remove commas from total_payment and convert it to a float
            total_payment = float(total_payment.replace(',', ''))
            customer_names.append(customer_name)
            total_payments.append(total_payment)

    # Step 3: DataFrame Creation
    # Create pandas DataFrame
    df = pd.DataFrame({
        'Customer Name': customer_names,
        'Total Payments ($)': total_payments
    })

    return df

In [ ]:
df_customer_payment = parse_customer_payment_response(response)

In [ ]:
df_customer_payment

,Customer Name,Total Payments ($)
0,Euro+ Shopping Channel,715738.98
1,Mini Gifts Distributors Ltd.,584188.24
2,"Australian Collectors, Co.",180585.07
3,Muscle Machine Inc,177913.95
4,"Dragon Souveniers, Ltd.",156251.03
5,"Down Under Souveniers, Inc",154622.08
6,"AV Stores, Co.",148410.09
7,"Anna's Decorations, Ltd",137034.22
8,Corporate Gift Ideas Co.,132340.78
9,"Saveley & Henriot, Co.",130305.35


In [ ]:
#Bar Plot for Total Payments by Customer Name

# Sort the DataFrame by 'Total Payments'
df_sorted = df_customer_payment.sort_values(by='Total Payments ($)', ascending=True)

fig = px.bar(df_sorted, y='Customer Name', x='Total Payments ($)',
             text='Total Payments ($)', title='Total Payments by Customer Name',
             color='Total Payments ($)', color_continuous_scale='Blugrn')

# Update the text on the bars
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

# Display the plot
fig.show()

In [ ]:
# Pie Chart for Share of Total Payments by Customer

# Sort the DataFrame by 'Total Payments'
df_sorted = df_customer_payment.sort_values(by='Total Payments ($)', ascending=True)

# Generate color indices
num_of_customers = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, num_of_customers).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]

# Create the pie chart
fig = px.pie(df_sorted, names='Customer Name', values='Total Payments ($)',
             title='Share of Total Payments by Customer',
             color_discrete_sequence=colors)
fig.show()


**Identify the top 10 most frequent customers**

In [ ]:
# List the top 10 most frequent customers?
query = input ("Enter question :")

Enter question :List the top 10 most frequent customers?


In [ ]:
response = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `customers, orders`
responded: The relevant tables for this query could be 'customers' and 'orders'. Let's check the schema of these tables.


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from 

In [ ]:
print(response)

The top 10 most frequent customers are:

1. Euro+ Shopping Channel with 26 orders
2. Mini Gifts Distributors Ltd. with 17 orders
3. Down Under Souveniers, Inc with 5 orders
4. Dragon Souveniers, Ltd. with 5 orders
5. Australian Collectors, Co. with 5 orders
6. Reims Collectables with 5 orders
7. Danish Wholesale Imports with 5 orders
8. Baane Mini Imports with 4 orders
9. Blauer See Auto, Co. with 4 orders
10. Kelly's Gift Shop with 4 orders


In [ ]:
def parse_customer_orders_response(data_str):
    # Step 1: Data Preprocessing
    # Extract lines containing customer and order information
    lines = [line.strip() for line in data_str.strip().split('\n') if 'orders' in line]

    # Step 2: Data Extraction
    # Extract customer names and their number of orders
    customer_names = []
    num_orders = []

    for line in lines:
        # Use regular expressions to capture customer names and number of orders
        match = re.search(r'^\d+\.\s(.*?)\swith\s(\d+)\sorders', line)
        if match:
            customer_name, orders = match.groups()
            customer_names.append(customer_name)
            num_orders.append(int(orders))

    # Step 3: DataFrame Creation
    # Create pandas DataFrame
    df = pd.DataFrame({
        'Customer Name': customer_names,
        'Number of Orders': num_orders
    })

    return df

In [ ]:
df_customer_orders = parse_customer_orders_response(response)

In [ ]:
df_customer_orders

,Customer Name,Number of Orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,"Down Under Souveniers, Inc",5
3,"Dragon Souveniers, Ltd.",5
4,"Australian Collectors, Co.",5
5,Reims Collectables,5
6,Danish Wholesale Imports,5
7,Baane Mini Imports,4
8,"Blauer See Auto, Co.",4
9,Kelly's Gift Shop,4


In [ ]:
#Bar Plot for Customer Orders by Customer Name

# Sort the DataFrame by 'Total Payments'
df_sorted = df_customer_orders.sort_values(by='Number of Orders', ascending=True)

fig = px.bar(df_sorted, y='Customer Name', x='Number of Orders',
             text='Number of Orders', title='Customer Orders by Customer Name',
             color='Number of Orders', color_continuous_scale='Blugrn')

# Update the text on the bars
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

# Display the plot
fig.show()

In [ ]:
# Pie Chart for Share of Orders by Customer

# Sort the DataFrame by 'Total Payments'
df_sorted = df_customer_orders.sort_values(by='Number of Orders', ascending=True)

# Generate color indices
num_of_customers = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, num_of_customers).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]

# Create the pie chart
fig = px.pie(df_sorted, names='Customer Name', values='Number of Orders',
             title='Share of Orders by Customer',
             color_discrete_sequence=colors)
fig.show()

**Analyze number of orders by productline**

In [ ]:
# List the number of orders by productline?
query = input ("Enter question :")

Enter question :List the number of orders by productline?


In [ ]:
response = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `orders, orderdetails, products`
responded: The relevant tables for this query seem to be 'orders', 'orderdetails', and 'products'. Let's check the schema of these tables to understand their structure and relationships.


CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	ord

In [ ]:
print(response)

Here is the number of orders by product line:

- Classic Cars: 1010 orders
- Motorcycles: 359 orders
- Planes: 336 orders
- Ships: 245 orders
- Trains: 81 orders
- Trucks and Buses: 308 orders
- Vintage Cars: 657 orders


In [ ]:
def parse_orders_by_productline_response(data_str):
    # Step 1: Data Preprocessing
    # Extract lines containing product line and order information
    lines = [line.strip() for line in data_str.strip().split('\n') if 'orders' in line]

    # Step 2: Data Extraction
    # Extract product lines and their number of orders
    product_lines = []
    num_orders = []

    for line in lines:
        # Use regular expressions to capture product lines and number of orders
        match = re.search(r'-\s(.*?):\s(\d+)\sorders', line)
        if match:
            product_line, orders = match.groups()
            product_lines.append(product_line)
            num_orders.append(int(orders))

    # Step 3: DataFrame Creation
    # Create pandas DataFrame
    df = pd.DataFrame({
        'Product Line': product_lines,
        'Number of Orders': num_orders
    })

    return df

In [ ]:
df_orders_by_productline = parse_orders_by_productline_response(response)

In [ ]:
df_orders_by_productline

,Product Line,Number of Orders
0,Classic Cars,1010
1,Motorcycles,359
2,Planes,336
3,Ships,245
4,Trains,81
5,Trucks and Buses,308
6,Vintage Cars,657


In [ ]:
#Bar Plot for Number of Orders by Productline

# Sort the DataFrame by 'Number of Orders'
df_sorted = df_orders_by_productline.sort_values(by='Number of Orders', ascending=False)

fig = px.bar(df_sorted, y='Number of Orders', x='Product Line',
             text='Number of Orders', title='Number of Orders by Productline',
             color='Number of Orders', color_continuous_scale='Blugrn')

# Update the text on the bars
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

# Display the plot
fig.show()


In [ ]:
# Donut Chart for Number of Orders by Productline

# Sort the DataFrame by 'Number of Orders'
df_sorted = df_orders_by_productline.sort_values(by='Number of Orders', ascending=False)

# Generate color indices
num_of_orders = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, num_of_orders).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]

# Create the Donut Chart
fig = px.pie(df_sorted, names='Product Line', values='Number of Orders', hole=0.5, title='Share of Orders by Product Line', color_discrete_sequence=colors)
fig.show()


In [ ]:
# Pie Chart for Number of Orders by Productline

# Sort the DataFrame by 'Number of Orders'
df_sorted = df_orders_by_productline.sort_values(by='Number of Orders', ascending=False)

# Generate color indices
num_of_orders = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, num_of_orders).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]

# Create the pie chart
fig = px.pie(df_sorted, names='Product Line', values='Number of Orders',
             title='Number of Orders by Product Line',
             color_discrete_sequence=colors)
fig.show()

In [ ]:
# Pandas profiling report
orders_by_productline_profile = ProfileReport(df_orders_by_productline, title='Number of Orders by Product Line', explorative=True)

In [ ]:
# profiling report notebook exploration
orders_by_productline_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

**Analyze sum of quantity ordered by year**

In [ ]:
# List the sum of quantity ordered by year?
query = input ("Enter question :")

Enter question :List the sum of quantity ordered by year?


In [ ]:
response = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `orders, orderdetails`
responded: The relevant tables for this query seem to be 'orders' and 'orderdetails'. Let's check the schema of these tables.


CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
10100	S18_2248	50	55.09	2
10100

In [ ]:
print(response)

The sum of quantity ordered by year is as follows:

- In 2003, the total quantity ordered was 36,439.
- In 2004, the total quantity ordered was 49,487.
- In 2005, the total quantity ordered was 19,590.


In [ ]:
def parse_quantity_ordered_by_year_response(data_str):
    # Step 1: Data Preprocessing
    # Extract lines that contain year and quantity information
    lines = [line.strip() for line in data_str.strip().split('\n') if 'total quantity ordered was' in line]

    # Step 2: Data Extraction
    # Extract years and their corresponding total quantities
    years = []
    quantities = []

    for line in lines:
        # Using regular expressions to capture year and total quantity
        match = re.search(r'In (\d{4}), the total quantity ordered was ([\d,]+)\.', line)
        if match:
            year, quantity = match.groups()
            years.append(int(year))
            quantities.append(int(quantity.replace(',', '')))  # Removing comma and converting to integer

    # Step 3: DataFrame Creation
    # Create a pandas DataFrame
    df = pd.DataFrame({
        'Year': years,
        'Total Quantity Ordered': quantities
    })

    return df

In [ ]:
df_quantity_ordered_by_year = parse_quantity_ordered_by_year_response(response)

In [ ]:
df_quantity_ordered_by_year

,Year,Total Quantity Ordered
0,2003,36439
1,2004,49487
2,2005,19590


In [ ]:
#Bar Plot for Total Quantity Ordered by Year

# Sort the DataFrame by 'Total Quantity Ordered'
df_sorted = df_quantity_ordered_by_year.sort_values(by='Total Quantity Ordered', ascending=False)

fig = px.bar(df_sorted, y='Total Quantity Ordered', x='Year',
             text='Total Quantity Ordered', title='Total Quantity Ordered by Year',
             color='Total Quantity Ordered', color_continuous_scale='Blugrn')

# Update the text on the bars
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside', width=[0.4]*len(df_sorted))

fig.update_xaxes(tickvals=[2003, 2004, 2005], ticktext=['2003', '2004', '2005'])

# Display the plot
fig.show()


In [ ]:
#Line graph showing trend of products ordered by Year

# Sort the DataFrame by 'Year'
df_sorted = df_quantity_ordered_by_year.sort_values(by='Year')

# Create the line graph
fig = px.line(df_sorted, x='Year', y='Total Quantity Ordered', title='Trend of Products Ordered by Year')

# Add points for each data point
fig.update_traces(mode='lines+markers')

# Update x-axis to only show specific tick labels (2003, 2004, 2005)
fig.update_xaxes(tickvals=[2003, 2004, 2005], ticktext=['2003', '2004', '2005'])

# Show the plot
fig.show()

In [ ]:
# Donut Chart for Total Quantity Ordered by Year

# Sort the DataFrame by 'Total Quantity Ordered'
df_sorted = df_quantity_ordered_by_year.sort_values(by='Total Quantity Ordered', ascending=False)

# Generate color indices
quantity_ordered = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, quantity_ordered).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]

# Create the Donut Chart
fig = px.pie(df_sorted, names='Year', values='Total Quantity Ordered', hole=0.5, title='Total Quantity Ordered by Year', color_discrete_sequence=colors)
fig.show()


**Identify top 10 performing products**

In [ ]:
# List the top 10 products by payments?
query = input ("Enter question :")

Enter question :List the top 10 products by payments?


In [ ]:
response = agent_executor.run(query)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `products, orderdetails, payments`
responded: The relevant tables for this query seem to be 'products', 'orderdetails', and 'payments'. Let's check the schema of these tables to understand their structure.


CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
1

In [ ]:
print(response)

Here are the top 10 products by payments:

1. 1992 Ferrari 360 Spider red: $276,839.98
2. 2001 Ferrari Enzo: $190,755.86
3. 1952 Alpine Renault 1300: $190,017.96
4. 2003 Harley-Davidson Eagle Drag Bike: $170,686.00
5. 1968 Ford Mustang: $161,531.48
6. 1969 Ford Falcon: $152,543.02
7. 1980s Black Hawk Helicopter: $144,959.91
8. 1998 Chrysler Plymouth Prowler: $142,530.63
9. 1917 Grand Touring Sedan: $140,535.60
10. 2002 Suzuki XREO: $135,767.03


In [ ]:
def parse_product_sales_response(data_str):
    # Use regex to find all occurrences of the pattern (product names and total sales)
    pattern = r"\d+\.\s(.*?):\s\$(\d{1,3}(?:,\d{3})*\.\d{2})"
    matches = re.findall(pattern, data_str)

    # Separate the matches into two lists: product_names and total_sales
    product_names = [match[0] for match in matches]
    total_sales = [float(match[1].replace(',', '')) for match in matches]

    # Create a pandas DataFrame
    df = pd.DataFrame({
        'Product Name': product_names,
        'Total Sales': total_sales
    })

    return df

In [ ]:
df_product_sales = parse_product_sales_response(response)

In [ ]:
df_product_sales

,Product Name,Total Sales
0,1992 Ferrari 360 Spider red,276839.98
1,2001 Ferrari Enzo,190755.86
2,1952 Alpine Renault 1300,190017.96
3,2003 Harley-Davidson Eagle Drag Bike,170686.00
4,1968 Ford Mustang,161531.48
5,1969 Ford Falcon,152543.02
6,1980s Black Hawk Helicopter,144959.91
7,1998 Chrysler Plymouth Prowler,142530.63
8,1917 Grand Touring Sedan,140535.60
9,2002 Suzuki XREO,135767.03


In [ ]:
#Bar Plot for the Top 10 Products by Total Sales

# Sort the DataFrame by 'Total Payments'
df_sorted = df_product_sales.sort_values(by='Total Sales', ascending=True)

fig = px.bar(df_sorted, y='Product Name', x='Total Sales',
             text='Total Sales', title='Top 10 Products by Total Sales',
             color='Total Sales', color_continuous_scale='Blugrn')

# Update the text on the bars
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

# Display the plot
fig.show()

In [ ]:
# Total Sales by Product in a Treemap

# Sort the DataFrame by 'Total Quantity Ordered'
df_sorted = df_product_sales.sort_values(by='Total Sales', ascending=True)

# Generate color indices
total_sales = len(df_sorted)
color_indices = np.linspace(0, len(px.colors.sequential.Turbo) - 1, total_sales).astype(int)

# Generate the color scale
colors = [px.colors.sequential.Turbo[i] for i in color_indices]



fig = px.treemap(df_sorted, path=['Product Name'], values='Total Sales',
                 title='Total Sales by Product in a Treemap', color_discrete_sequence=colors)
fig.show()


In [ ]:
# Pandas profiling report
product_sales_profile = ProfileReport(df_product_sales, title='Total Sales by Product', explorative=True)

In [ ]:
# profiling report notebook exploration
product_sales_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]